# Local Model Serving with MLflow

In [2]:
import httpx 
import json
import mlflow
import pandas as pd 
import numpy as np
from examples.digit_recognition.data import get_train_val_test_data
from examples.digit_recognition.data import transform_to_image

from mlflow_for_ml_dev.src.utils.folder_operations import get_project_root

# set mlflow tracking uri
mlflow.set_tracking_uri(uri=(get_project_root() / 'mlruns').as_uri())

# Scoring Digit recognizer

```shell
mlflow models serve --options
```

To run the code below make sure you deploy the model using.

`poetry run mlflow models serve --model-uri models:/Digit_Recognition_Model@production --env-manager local`

In [3]:
_, _, x_test, _, _, y_test = get_train_val_test_data()
x_test = transform_to_image(x_test)

url = "http://127.0.0.1:5000/invocations"
n_samples = 1
samples = x_test[0:n_samples]

payload = {
    "instances": {"image_input": samples.tolist()},
}

headers = {"Content-Type": "application/json"}
response = httpx.post(url, data=json.dumps(payload), headers=headers)

if response.status_code == 200:
    predictions = response.json().get("predictions")
    pred = np.argmax(predictions, axis=-1)
    print(pd.DataFrame({"predictions": pred, "y_test": y_test[0:n_samples]}))
    
else:
    raise Exception(f"Error: {response.status_code} - {response.text}")

       predictions  y_test
14150            9       9


## Signature Validation

In [9]:
_, _, x_test, _, _, y_test = get_train_val_test_data()
x_test = transform_to_image(x_test)

url = "http://127.0.0.1:5000/invocations"
n_samples = 1
samples = x_test[0:n_samples]

payload = {
    "instances": {"image_inputs": samples.tolist()}, # invalid input name
    # "instances": {"image_input": samples.tolist()}, # valid input name
}

headers = {"Content-Type": "application/json"}
response = httpx.post(url, data=json.dumps(payload), headers=headers)

In [10]:
print(response.json()["message"])

Failed to predict data '{'image_inputs': array([[[[  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.]],

        [[  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.],
         [  0.]],

   

## Changing the input shape

In [13]:
_, _, x_test, _, _, y_test = get_train_val_test_data()
x_test = transform_to_image(x_test)

url = "http://127.0.0.1:5000/invocations"
n_samples = 1
samples = x_test[0:n_samples]
samples = samples.reshape(28, 28, 1) # invalid input shape
payload = {
    "instances": {"image_input": samples.tolist()}, # invalid input name
}

headers = {"Content-Type": "application/json"}
response = httpx.post(url, data=json.dumps(payload), headers=headers)

In [12]:
print(response.json()["message"])

Failed to predict data '{'image_input': array([[[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0

## Using Inputs Field as Key for the payload

In [14]:
_, _, x_test, _, _, y_test = get_train_val_test_data()
x_test = transform_to_image(x_test)

url = "http://127.0.0.1:5000/invocations"
n_samples = 1
samples = x_test[0:n_samples]
payload = {
    "inputs": {"image_input": samples.tolist()}, # invalid input name
}

headers = {"Content-Type": "application/json"}
response = httpx.post(url, data=json.dumps(payload), headers=headers)

In [15]:
response.json()

{'predictions': [[7.647104212082922e-06,
   2.0330107375343687e-08,
   3.468953764240723e-06,
   1.9539262211765163e-05,
   4.324278506828705e-06,
   1.0750986803031992e-05,
   2.3375855917606714e-08,
   0.0001128212534240447,
   0.0001029914419632405,
   0.9997383952140808]]}